Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [ ]:
from numpy import random
from random import choices
from tqdm import tqdm

import lab9_lib
import numpy as np
import copy
import matplotlib.pyplot as plt

In [ ]:
POP_SIZE = 10          # Number of parents
N_OFFSPRINGS = 30      # Number of children
N_GENERATIONS = 10000   # Number of generations
TOP_K = 30              # Number of parents from which generating the offsprings
SIZE = 1000               # Size of the genome
PROBLEM_SIZE = 5       # Size of the problem

In [ ]:
def generate_random_individual(): 
    ind = random.choice([0, 1], size=SIZE)
    return ind

In [ ]:
fitness = lab9_lib.make_problem(2)
for n in range(10):
    ind = generate_random_individual()
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

# Ideas

## Tricks and ideas

- Save configurations + fitness in a dictionary to avoid repetitions
- Create a CNN+FC to, first extract the genome information (necessary spatial information) and then predict the associated fitness


## Island

- Generate K islands, evolve them independently and, each N mutations, let one individual from each travel to others (one individual can travel to multiple islands?). How to choose the individual?


## Mutation

- Change the bit with a given probability. How many bits to mutate?


## Save the best

- Elitism: copy the champion into the offsprings without modifications
- Valhalla: keep the best apart and, from time to time, put them back in the population


## Recombination

- Implement one cuts/two cuts/n cuts
- Do it choosing the idx through a Gaussian


## Parent selection (semi stochastic)

- Try roulette (linearized proportional to fitness, trouble with big populations)
- Try tournament (increase size to increase pressure)


## Survival selection (fully deterministic)

- Filter only the top K fittest individuals

# Our Solution

## V1.0

The simplest approach.

In [ ]:
def mutate_bit(individual : np.array) -> np.array:
    
    idx = np.random.randint(0, len(individual))
    mutated_individual = copy.deepcopy(individual)
    mutated_individual[idx] = int(not(bool(mutated_individual[idx])))
    return mutated_individual


def roulette_selection(individuals : np.array, fitness : np.array) -> np.array:

    # Select top K fittest individuals and return N_OFFSPRINGS couple of individuals.
    parents = choices(individuals, weights=np.array(fitness)/sum(fitness), k=2)
    return (parents[0], parents[1]) # Return a tuple of two individuals.


def one_cut_xover(couple : tuple) -> np.array:

    idx = np.random.randint(1, len(couple[0]) - 1)
    return np.concatenate((couple[0][:idx], couple[1][idx:]))

In [ ]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)
pop = np.array([generate_random_individual() for _ in range(POP_SIZE)])

In [ ]:
fitnessses = np.array([fitness(elem) for elem in pop])
parents = np.array([roulette_selection(pop, fitnessses) for _ in range(N_OFFSPRINGS)])

for _ in tqdm(range(N_GENERATIONS)):
        
    recombinated_offsprings = np.array([one_cut_xover(couple) for couple in parents])
    mutated_offsprings = np.array([mutate_bit(ind) for ind in recombinated_offsprings])
    fit = np.array([fitness(ind) for ind in mutated_offsprings])
    survived = mutated_offsprings[np.argsort(fit) < POP_SIZE]
    if (fit > 0.95).any():
        break
    fitnessses = np.array([fitness(elem) for elem in pop])
    parents = np.array([roulette_selection(pop, fitnessses) for _ in range(N_OFFSPRINGS)])

In [ ]:
mutated_fitness = list(map(fitness, mutated_offsprings))
ordered_mutated = [_ for _, x in sorted(zip(mutated_offsprings, mutated_fitness), key=lambda pair: pair[1], reverse=True)]

for sur in ordered_mutated:
    print(f"{''.join(str(g) for g in sur)}: {fitness(sur):.2%}")

In [ ]:
fitness.calls

# V1.1

In [ ]:
def reproduce1(couple : tuple) -> np.array:
    new_ind = copy.deepcopy(couple[0])
    for i in range(len(couple[0])):
        # mucho faster cosi
        if couple[0][i] == couple[1][i]:
            continue
        gene_giver = random.choice([0,1])
        new_ind[i] = couple[gene_giver][i]
    return new_ind

In [ ]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)
pop = np.array([generate_random_individual() for _ in range(POP_SIZE)])

In [ ]:
fitnessses = np.array([fitness(elem) for elem in pop])
parents = np.array([roulette_selection(pop, fitnessses) for _ in range(N_OFFSPRINGS)])

for _ in tqdm(range(N_GENERATIONS)):
        
    recombinated_offsprings = np.array([reproduce1(couple) for couple in parents])
    mutated_offsprings = np.array([mutate_bit(ind) for ind in recombinated_offsprings])
    fit = np.array([fitness(ind) for ind in mutated_offsprings])
    survived = mutated_offsprings[np.argsort(fit) < POP_SIZE]
    if (fit > 0.95).any():
        break
    fitnessses = np.array([fitness(elem) for elem in pop])
    parents = np.array([roulette_selection(pop, fitnessses) for _ in range(N_OFFSPRINGS)])

In [ ]:
mutated_fitness = list(map(fitness, mutated_offsprings))
ordered_mutated = [_ for _, x in sorted(zip(mutated_offsprings, mutated_fitness), key=lambda pair: pair[1], reverse=True)]

for sur in ordered_mutated:
    print(f"{''.join(str(g) for g in sur)}: {fitness(sur):.2%}")

In [ ]:
fitness.calls

# V1.3

Memoization of the fitness function.

In [ ]:
memoization = {}
def memoization(individual: np.array) -> float:
    already_seen = False
    if memoization.get(individual.tobytes()) is None:
        already_seen = True
        memoization[individual.tobytes()] = fitness(individual)
    return (memoization[individual.tobytes()], already_seen)


In [ ]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)
pop = np.array([generate_random_individual() for _ in range(POP_SIZE)])

In [ ]:
parents = roulette_selection(TOP_K, pop, [fitness(ind) for ind in pop])

for _ in tqdm(range(N_GENERATIONS)):
        
    recombinated_offsprings = map(reproduce1, parents)
    mutated_offsprings = np.array(list(map(mutate_bit, recombinated_offsprings)))
    fit = np.array(list(map(memoization, mutated_offsprings)))
    survived = mutated_offsprings[np.argsort(fit) < POP_SIZE]
    if (fit > 0.95).any():
        break
    parents = roulette_selection(TOP_K, survived, [fitness(ind) for ind in survived])

In [ ]:
mutated_fitness = list(map(fitness, mutated_offsprings))
ordered_mutated = [_ for _, x in sorted(zip(mutated_offsprings, mutated_fitness), key=lambda pair: pair[1], reverse=True)]

for sur in ordered_mutated:
    print(f"{''.join(str(g) for g in sur)}: {fitness(sur):.2%}")

In [ ]:
fitness.calls()

## V1.5
Basic implementation of the Valhalla loading the champions at every epoch.

In [ ]:
POP_SIZE = 100          # Number of parents
N_OFFSPRINGS = 300      # Number of children
N_GENERATIONS = 10000   # Number of generations
TOP_K = 30              # Number of parents from which generating the offsprings
EPOCH = 30              # Number of generation to reintroduce the best fitting
N_CHAMPIONS = 4         # Number of champions that will rest in the Valhalla
SIZE = 50               # Size of the genome


pop = []
for _ in range(POP_SIZE):
    pop.append(choices([0, 1], k=SIZE))

In [ ]:
parents = roulette_selection(TOP_K, pop, [fitness(ind) for ind in pop])

# 3 is the number of N_CHAMPIONS that can be stored simultaneously. For example,
# if N_CHAMPIONS == 4, the valhalla can store up to 12 champions.
valhalla = np.empty(shape = (3 * N_CHAMPIONS, SIZE), dtype=np.int16)   
max_fit = []
avg_fit = []

for _ in range(N_GENERATIONS):
    
    recombinated_offsprings = map(one_cut_xover, parents)
    mutated_offsprings = np.array(list(map(mutate_bit, recombinated_offsprings)))
    fit = np.array(list(map(fitness, mutated_offsprings)))
    
    survived = mutated_offsprings[np.argsort(fit) < POP_SIZE]
    max_fit.append(max(fit))        # Simple statistics
    avg_fit.append(np.mean(fit))
    
    if (fit > 0.95).any():
        break
    
    # Every 10 generations, pick the champions and put them in a buffer, which, once full
    # (size equal to 3 * N_CHAMPIONS) keeps the most recent 2 * N_CHAMPIONS and substitutes
    # the oldest 1 * N_CHAMPIONS with the ones just computed.
    if _ % 10 == 0 and _ != 0:
        champions = mutated_offsprings[np.argsort(fit) < N_CHAMPIONS]
        start = (((_ - 1) % EPOCH) // 10) * 4
        end = (((_ - 1) % EPOCH) // 10) * 4 + 4
        valhalla[start:end, :] = mutated_offsprings[np.argsort(fit) < N_CHAMPIONS]

    # Every epoch, load the valhalla e compute the parents.
    if _ % EPOCH == 0 and _ != 0:
        survived = np.concatenate((survived, valhalla))

    # Introduce some randomicity in order to inject champions from time to time.
    if random() < 0.5 and (_ % EPOCH == 0 and _ != 0):
        survived = np.concatenate((survived, choices(valhalla, k = 4)))

    parents = roulette_selection(TOP_K, survived, [fitness(ind) for ind in survived])
    print(_)
    # print(_)

In [ ]:
mutated_fitness = list(map(fitness, mutated_offsprings))
ordered_mutated = [_ for _, x in sorted(zip(mutated_offsprings, mutated_fitness), key=lambda pair: pair[1], reverse=True)]

for sur in ordered_mutated:
    print(f"{''.join(str(g) for g in sur)}: {fitness(sur):.2%}")

In [ ]:
plt.plot(range(len(avg_fit)), avg_fit)

## V2.0

In [ ]:
N_ISLANDS = 5           # Number of different populations
POP_SIZE = 100          # Number of parents
N_OFFSPRINGS = 300      # Number of children
N_GENERATIONS = 10000   # Number of generations
TOP_K = 30              # Number of parents from which generating the offsprings
EPOCH = 50              # Number of generation to reintroduce the best fitting.

globe = {f'island{k}' : [] for k in range(N_ISLANDS)}

for n in range(N_ISLANDS):
    for _ in range(POP_SIZE):
        globe[f'island{n}'].append(choices([0, 1], k=30))
        
italy = globe['island0']

In [ ]:
def mutate_bits(individual : list) -> list:
    '''
    Mutate every bit with a given probability
    '''
    # TODO
    idx = np.random.randint(0, len(individual), size = len(individual))
    mutated_individual = copy.deepcopy(individual)
    np.random.random(size = len(individual))
    if np.random.random() < 0.9:
        mutated_individual[idx] = int(not(bool(mutated_individual[idx])))
    return mutated_individual
